# Groupby

`pd.DataFrame.groupby` is a very useful tool, but sometimes working with it can be a bit confusing. So in this page I want to pay more attention to some functions and cases.

The most useful page for learning is <a href="https://pandas.pydata.org/docs/reference/groupby.html">GroupBy object in pandas documentation</a>.

# `Apply` - combine results

<a href="https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.apply.html#pandas.core.groupby.DataFrameGroupBy.apply">Pandas documentation about apply function.</a>

## Basic idea

The peculiarity of this method is that it uses `pandas.DataFrame` as the input for the aggregation function.

The following example shows this: `example_funtion' just prints the input and it always prints a DataFrame for each "A" variable option.

In [40]:
df = pd.DataFrame({'A': ['a', 'a', 'b', 'b', 'c', 'c'],
                   'B': [2, 1, 3, 4, 6, 5],
                   'C': [10, 20, 30, 40, 50, 60]})

def example_funtion(subdf):
    print("=========")
    print(subdf)
    return 5

res = df.groupby("A")[["A", "B", "C"]].apply(example_funtion)

   A  B   C
0  a  2  10
1  a  1  20
   A  B   C
2  b  3  30
3  b  4  40
   A  B   C
4  c  6  50
5  c  5  60


## Use case

So it's perfect for cases where you need to get, for each variant of variable A, some value of variable C conditioned on the value of variable B.

In particular, the following example shows how to obtain for each option of "A" the "C" value corresponding to the minimum "B" value.

- For `"A" == "a"` I got `"C" == 20`, because it corresponds to `"B"== 1`, which is the minimum for every `"A" == "a"`;
- For `"A" == "b"` I got `"C" == 30`, because it corresponds to `"B"== 3`, which is the minimum for every `"A" == "b"`;
- For `"A" == "c"` I got `"C" == 60`, because it corresponds to `"B"== 5`, which is the minimum for every `"A" == "c"`.

In [34]:
from IPython.display import HTML
import pandas as pd
df = pd.DataFrame({'A': ['a', 'a', 'b', 'b', 'c', 'c'],
                   'B': [2, 1, 3, 4, 6, 5],
                   'C': [10, 20, 30, 40, 50, 60]})

display(HTML("<b>Initial frame</b>"))
display(df)

result = df.groupby("A")[["B", "C"]].apply(
    lambda subset: subset.loc[subset["B"].idxmin(), "C"]
)
display(HTML("<strong>Result</strong>"))
result.rename("C").to_frame()

,A,B,C
0,a,2,10
1,a,1,20
2,b,3,30
3,b,4,40
4,c,6,50
5,c,5,60


,C
A,
a,20
b,30
c,60


## vs `agg`/`transform`

Other common functions may seem useless because this function can do everything they can. However, according to the <a href="https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.apply.html#pandas.core.groupby.DataFrameGroupBy.apply">pandas documentation</a>, they may work a little faster. I have not been able to test this yet.